In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from dataclasses import dataclass, asdict
import re
import sqlite3
import json
from pickling_cookies import pickle_thing, unpickle_thing
from prefect import task, Flow, Parameter

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
opts = Options()
opts.set_headless()


<ipython-input-4-a31150cfc6c0>:5: DeprecationWarning: use setter for headless property instead of set_headless
  opts.set_headless()


In [7]:
url ="https://www.goodreads.com/user/show/43182754-crawford-collins"
driver = webdriver.Firefox()
driver.get(url)
shelfContainer = driver.find_elements_by_class_name("userShowPageShelfListItem")
shelves = [grab_shelf(ii) for ii in shelfContainer]
#driver.close()
shelves = [asdict(i) for i in shelves]

In [18]:
?pickle_thing

In [11]:
cookies = driver.get_cookies()
cookies

[{'name': 'ccsid',
  'value': '386-1325278-1730327',
  'path': '/',
  'domain': 'www.goodreads.com',
  'secure': False,
  'httpOnly': False,
  'expiry': 2214621008},
 {'name': 'locale',
  'value': 'en',
  'path': '/',
  'domain': 'www.goodreads.com',
  'secure': False,
  'httpOnly': False},
 {'name': '__utmc',
  'value': '250562704',
  'path': '/',
  'domain': '.goodreads.com',
  'secure': False,
  'httpOnly': False},
 {'name': '__utmt',
  'value': '1',
  'path': '/',
  'domain': '.goodreads.com',
  'secure': False,
  'httpOnly': False,
  'expiry': 1583469628},
 {'name': '__qca',
  'value': 'P0-892440638-1583469029019',
  'path': '/',
  'domain': '.goodreads.com',
  'secure': False,
  'httpOnly': False,
  'expiry': 1617337829},
 {'name': '__gads',
  'value': 'ID=257d8a500529f64d:T=1583469026:S=ALNI_MYY8XLYvx_MEoNJnQ8c4n-rW5VBvQ',
  'path': '/',
  'domain': '.goodreads.com',
  'secure': False,
  'httpOnly': False,
  'expiry': 1646541026},
 {'name': 'never_show_interstitial',
  'value': 

In [12]:
cookies.save()

AttributeError: 'list' object has no attribute 'save'

In [16]:
cookies=pickle_thing(driver.get_cookies(),"cookies")

In [6]:
@dataclass
class Shelf:
    name: str
    num_books : int
    link: str

@dataclass
class Book:
    title: str
    author: str
    link: str
    isbn13: str
    shelf_url : str

@task
def construct_shelves(profile_url):
    driver = webdriver.Firefox(options=opts)
    driver.get(profile_url)
    shelfContainer = driver.find_elements_by_class_name("userShowPageShelfListItem")
    shelves = [grab_shelf(ii) for ii in shelfContainer]
    driver.close()
    shelves = [asdict(i) for i in shelves]
    return shelves

def grab_shelf(shelf):
    name = re.split(r'\([0-9]+\)',shelf.text)[0].replace("(","").replace(")","").strip()
    num_books = int(re.findall(r'\([0-9]+\)',shelf.text)[0].replace("(","").replace(")",""))
    link = shelf.get_attribute("href")
    return Shelf(name,num_books,link)

In [4]:
url ="https://www.goodreads.com/user/show/43182754-crawford-collins"

In [5]:
#c = construct_shelves(url)

In [22]:
@task
def add_shelf_to_db(shelf_dict):
    
    shelf = Shelf(
        name=shelf_dict["name"],
        num_books=shelf_dict["num_books"],
        link = shelf_dict["link"]
    )
    driver = webdriver.Firefox(options=opts)
    cookies = unpickle_thing("cookies")
    for ii in cookies:
        driver.add_cookie(ii)
    
    driver.get(shelf.link)
    driver.find_elements_by_id("shelfSettingsLink")[0].click()
    driver.find_elements_by_name("shelf[display_fields][isbn13]")[0].click()
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        # Calculate new scroll height and compare with last scroll height
        dd =driver.find_elements_by_id("infiniteStatus")

        frac,total = [int(jj) for jj in dd[0].text.replace(" loaded","").split(" of ")]
        if frac == total:
            break
            
    titles = driver.find_elements_by_class_name("field.title")
    author = driver.find_elements_by_class_name("author")
    isbn13 = driver.find_elements_by_class_name("isbn13")

    conn = sqlite3.connect('books.db')
    curs = conn.cursor()

    if len(titles) == len(author) == len(isbn13):
        for i in range(1,len(titles)):
            bb = Book(
                titles[i].text,
                author[i].text,
                titles[i].find_elements_by_class_name("value")[0].find_element_by_tag_name("a").get_attribute("href"),
                isbn13[i].text,
                shelf_url = url)
            curs.execute(f"INSERT into books values {astuple(bb)}")
    conn.commit()
    curs.close()
        
    driver.close()       

In [23]:
url ="https://www.goodreads.com/user/show/43182754-crawford-collins"

with Flow("shelves_to_db") as flow:
    user_profile_url = Parameter("user_profile_url")
    shelves = construct_shelves(user_profile_url)
    add_shelf_to_db.map(shelf_dict = shelves)
    
flow.run(
    user_profile_url = url
)

[2020-03-06 04:45:33,861] INFO - prefect.FlowRunner | Beginning Flow run for 'shelves_to_db'
[2020-03-06 04:45:33,879] INFO - prefect.FlowRunner | Starting flow run.
[2020-03-06 04:45:33,925] INFO - prefect.TaskRunner | Task 'user_profile_url': Starting task run...
[2020-03-06 04:45:34,025] INFO - prefect.TaskRunner | Task 'user_profile_url': finished task run for task with final state: 'Success'
[2020-03-06 04:45:34,075] INFO - prefect.TaskRunner | Task 'construct_shelves': Starting task run...
[2020-03-06 04:46:13,348] INFO - prefect.TaskRunner | Task 'construct_shelves': finished task run for task with final state: 'Success'
[2020-03-06 04:46:13,416] INFO - prefect.TaskRunner | Task 'add_shelf_to_db': Starting task run...
[2020-03-06 04:46:13,511] INFO - prefect.TaskRunner | Task 'add_shelf_to_db[0]': Starting task run...
[2020-03-06 04:46:23,410] ERROR - prefect.TaskRunner | Unexpected error: InvalidCookieDomainException('Document is cookie-averse', None, None)
Traceback (most rece

<Failed: "Some reference tasks failed.">

In [8]:
def sqlite_helper(query_str,db_name):
    conn = sqlite3.connect(db_name)
    curs = conn.cursor()
    curs.execute(query_str)
    output =  curs.fetchall()
    conn.commit()
    conn.close()
    return output

In [9]:
query_str = "SELECT Count(*) from books"
db_name = 'books.db'
output = sqlite_helper(query_str,db_name)

In [10]:
output

[(207,)]